In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

In [ ]:
# 깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

import pandas as pd
import numpy as np


#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
data_text = np.load('/content/drive/MyDrive/Colab Notebooks/Data/sttEmotionTrainTest.npz')
data_params = np.load('/content/drive/MyDrive/Colab Notebooks/Data/speechEmotion1.npz')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/data/sttEmotion.npz'

In [ ]:
X_train_text=data_text['X_train']
y_train_text=data_text['y_train']
X_test_text=data_text['X_test']
y_test_text=data_text['y_test']

X_train_params=data_params['X_train']
y_train_params=data_params['y_train']
X_test_params=data_params['X_test']
y_test_params=data_params['y_test']


In [ ]:
import re

def preprocess_sentence_kr(w):
    w = w.strip()
    w = re.sub(r"[^0-9가-힣?.!,¿]+", " ", w) # \n도 공백으로 대체해줌
    w = w.strip()
    return w


In [ ]:
DATA_COLUMN = "document"
LABEL_COLUMN = "label"

In [ ]:
trainSet_text=[]
for i in range(len(X_train_text)):
    trainSet_text.append(preprocess_sentence_kr(X_train_text[i]))

In [ ]:
testSet_test=[]
for i in range(len(y_test_text)):
    testSet_test.append(preprocess_sentence_kr(X_test_text[i]))

In [ ]:
train_text = pd.DataFrame({DATA_COLUMN:trainSet_text,LABEL_COLUMN:y_train_text})
test_text = pd.DataFrame({DATA_COLUMN:testSet_test,LABEL_COLUMN:y_test_text})

In [ ]:
def DatasetList(dataFrame):
    data_list = []
    for q, label in zip(dataFrame[DATA_COLUMN], dataFrame[LABEL_COLUMN])  :
        data = []
        data.append(q)
        data.append(str(label))

        data_list.append(data)
    return data_list

In [ ]:
train_text = DatasetList(train_text)
test_text=DatasetList(test_text)

In [ ]:
# BERT 모델에 들어가기 위한 dataset을 만들어주는 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair,paramsData):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]
        # self.params =[np.int32(i) for i in paramsData]
    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
# 숫자를 조정해도 무방
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 2
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
#패딩
data_train_text = BERTDataset(train_text, 0, 1, tok, max_len, True, False,X_train_params)
data_test_text = BERTDataset(test_text, 0, 1, tok, max_len, True, False,X_test_params)

In [ ]:
#torch 형식의 dataset
train_dataloader_text = torch.utils.data.DataLoader(data_train_text, batch_size=batch_size, num_workers=5)
test_dataloader_text = torch.utils.data.DataLoader(data_test_text, batch_size=batch_size, num_workers=5)

In [ ]:
X_train_params = X_train_params.reshape((453965,180,1,1))
X_test_params = X_test_params.reshape((56248,180,1,1))

In [ ]:
X_train_params_tensor = torch.utils.data.DataLoader(X_train_params, batch_size=batch_size, num_workers=5)
X_test_params_tensor = torch.utils.data.DataLoader(X_test_params, batch_size=batch_size, num_workers=5)

In [ ]:
from sklearn.utils.extmath import softmax
# Kobert _ CNN 통합 학습모델
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

        self.cnn = nn.Sequential(
            
            nn.Conv2d(in_channels=180,out_channels=32,kernel_size=2,padding='same'),  # 인풋 채널 갯수
            nn.BatchNorm2d(32),
            nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Dropout(p=0.2),
            
           
            nn.Conv2d(in_channels=32,out_channels=768,kernel_size=3,padding='same'),
            nn.BatchNorm2d(768),
            nn.ReLU(),
            # nn.MaxPool1d(kernel_size=2,stride=2),
            nn.Dropout(p=0.2),
            
            
            
            # nn.Conv1d(in_channels=16, out_channels=16, kernel_size=4,padding='same'),
            # nn.BatchNorm1d(16),
            # nn.ReLU(),
            # nn.MaxPool1d(kernel_size=2,stride=2),
            # nn.Dropout(p=0.2),

            # nn.Conv1d(in_channels=16, out_channels=16, kernel_size=4,padding='same'),
            # nn.BatchNorm1d(16),
            # nn.ReLU(),
            # nn.MaxPool1d(kernel_size=2,stride=2),
            # nn.Dropout(p=0.2),
            
           
           
            nn.Flatten(),
                
        )
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids,cnnData):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
            out_bert =nn.functional.softmax(out)
            out_cnn =nn.functional.softmax(self.cnn(cnnData))
            out_tatal = torch.cat((out_bert,out_cnn),dim=0)



            
        return self.classifier(out_tatal)

In [ ]:
model = BERTClassifier(bertmodel,dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
# 학습모델 학습
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, ((token_ids, valid_length, segment_ids, label),X_train_param) in enumerate(tqdm_notebook(zip(train_dataloader_text,X_train_params_tensor))): 
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        X_train_param=X_train_param.float().to(device)
      
        out = model(token_ids, valid_length, segment_ids,X_train_param)                                                              ##
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, ((token_ids, valid_length, segment_ids, label),X_test_param) in enumerate(tqdm_notebook(test_dataloader_text),X_test_params_tensor): #
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        X_test_param=X_test_param.long().to(device)
        label = label.float().to(device)

        out = model(token_ids, valid_length, segment_ids,X_train_param)                                                        ##
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))